In [94]:
import pandas as pd
import numpy as np
import re
import regex

The original dataset

In [117]:
fix_path = r"M:\Python\mocnhien\dataset\raw_disease_updated.csv"
df = pd.read_csv(fix_path)

In [118]:
df = df.dropna(subset=['Department', 'Symptom', 'Category'])

CleanText

As the sentences distribution are messy in some way. The main here idea is to first, remove all dots, truncate all trailing spaces. Then by the 
logic that all dot must followed by an UpperCased letter, except for the first letter in the sentence 


In [ ]:
# Test function before applying
text = "A.A.A. ."
re.sub(pattern=r"\.", repl=" ", string=text)

In [119]:
def cleanText(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(pattern=r"\.", repl=" ", string=text) # Remove dots
    text = re.sub(pattern=r"[~!@#$%^&*()<>?,/'\";:\{\[\]\}+=\_\-\*\+]", repl="", string=text) # Remove puncs
    text = re.sub(r'[✍☎⌨].*$', '', text) # Remove consultation-related spammy info
    text = re.sub(r'(?:[A-ZĐ]{1}(?:\s*\.\s*)){3,}[A-ZĐ]{1}', '', text)# Remove spammy letters like C . H . A . T . Z . A . L . O
    
    # Filter some noise by hand 
    text = re.sub(pattern=r"(Bác sĩ tham vấn thông tin và hỗ trợ khám bệnh)", repl=" ", string=text)
    text = re.sub(pattern=r"(Đội ngũ bác sĩ chúng tôi luôn sẵn sàng nghe bạn chia sẻ hãy liên hệ 1900 1246 hoặc hotline 0886006167)", repl=" ", string=text)

    # Normalize sentences
    text = regex.sub(r'(\p{Lu})', r'. \1', text) # Adding dot before uppercased letters
    text = re.sub(pattern=r"(  +)", repl=" ", string=text) # Remove trailing spaces

    
    return text

In [120]:
df['Symptom'] = df['Symptom'].apply(lambda x:cleanText(x))

In [122]:
# Remove the leading dot
prefix_dot = df[df['Symptom'].str[0] == '.'].index

for idx in prefix_dot:
    df.loc[idx, 'Symptom'] = df.loc[idx, 'Symptom'].replace(".", "", 1).strip()

In [100]:
df.loc[14]['Symptom']

'. Người bị bệnh tâm thần phân liệt thường gặp nhiều vấn đề về tư duy nhận thức hành vi hay tình cảm . . Các dấu hiệu và triệu chứng của bệnh rất đa dạng nhưng nhìn chung thường có ảo tưởng ảo giác hoặc rối loạn tư duy và hành vi vô tổ chức . Ảo tưởng những niềm tin sai lầm không dựa trên thực tế . . Ví dụ nghĩ rằng bản thân đang bị hãm hại ai đó đang chỉ trích công kích mình nghĩ mình có năng lực đặc biệt hoặc nổi tiếng hay nhầm tưởng người khác có tình cảm với mình hay một thảm hoạ lớn sắp xảy ra . Ảo tưởng thường gặp ở hầu hết người bệnh . Ảo giác thường liên quan đến nhìn thấy hoặc nghe những thứ không tồn tại . . Ao giác có thể xảy ra ở bất kỳ giác quan nào nhưng nghe thấy tiếng nói lạ là ảo giác thường gặp nhất ảo giác thị giác . . Rối loạn tư duy . . Rối loạn tư duy là hệ quả từ sự rối loạn ngôn ngữ . . Rối loạn ngôn ngữ làm giảm sút hiệu quả giao tiếp vì câu trả lời của người bệnh chỉ đáp ứng được một phần hoặc hoàn toàn không liên quan gì đến câu hỏi của đối phương . . Ngoài r

In [126]:
[x for x in df['Symptom']]

['nếu ai đó có những dấu hiệu sau đây thì rất có thể họ đang có nguy cơ mắc bệnh ảo giác để hiểu rõ hơn về các triệu chứng cũng như nhận diện chính xác căn bệnh mình đang mắc phải bạn nên tham khảo một số thông tin sau nói ra ý tưởng liên quan hay suy nghĩ về những sự việc hay vật gì đó có ý nghĩa khác lạ phi lý thay đổi cách thức biểu lộ một việc gì tra cứu thông tin xem thêm về triệu chứng thờ ơ thường xuyên cảm nhận đang có điều gì phi thực tế xem thêm về rối loạn cảm xúc xuất hiện nhiều hành vi khác lạ rối loạn về cảm xúc như cảm giác hai chiều hoặc cảm xúc trái ngược người bệnh có thể bị căng thẳng mất ngủ đờ đẫn không muốn ăn uống hay tham gia bất kỳ các hoạt động gì thường xuyên ngồi hay nằm một chỗ xem thêm về triệu chứng mất ngủ quan tâm quá mức với ý nghĩ bất thường không có trong thực tế biểu lộ dửng dưng thờ ơ không biết cùng vui hay cùng buồn với mọi người ăn ngủ thất thường bệnh nhân thường xuyên bị mất ngủ khó ngủ hay đảo giấc cảm giác chán ăn hoặc ăn uống không đúng gi

Concat duplicate sentences

+ Removing duplicated sentences using set. Consider lowercasing all letters for clearer data :>

In [124]:
def concatSymptom(text):
   sentences = set([text.strip().lower() for text in text.split(".")])
   return ' '.join([sentence for sentence in sentences if sentence is not None])

In [125]:
df['Symptom'] = df['Symptom'].apply(lambda x: concatSymptom(x))

Unify Label

In [ ]:
df['Department'].value_counts()

In [ ]:
df['Category'].value_counts()

In [127]:

def LabelUnifier(label):
    departments = {
    "Thần kinh - Tâm thần": [
        "Thần kinh", "Nội thần kinh", "Ngoại thần kinh", "Tâm thần", "Tâm lý"
    ],
    "Hô hấp": [
        "Hô hấp"
    ],
    "Truyền nhiễm": [
        "Truyền nhiễm"
    ],
    "Tiết niệu - Nam khoa": [
        "Tiết niệu", "Thận tiết niệu", "Nam khoa", "Nam học"
    ],
    "Nội tiết": [
        "Nội tiết", "Nội tiết chuyển hóa"
    ],
    "Tiêu hóa - Gan mật": [
        "Tiêu hóa", "Tiêu hóa - Gan mật"
    ],
    "Tim mạch - Mạch máu": [
        "Tim mạch", "Mạch máu"
    ],
    "Ung thư": [
        "Ung thư", "Ung bướu"
    ],
    "Di truyền": [
        "Di truyền"
    ],
    "Răng hàm mặt": [
        "Răng hàm mặt"
    ],
    "Tai mũi họng": [
        "Tai mũi họng"
    ],
    "Da liễu": [
        "Da liễu", "Da tóc móng"
    ],
    "Nhãn khoa": [
        "Mắt", "Nhãn khoa"
    ],
    "Cơ xương khớp": [
        "Cơ xương khớp"
    ],
    "Dị ứng": [
        "Dị ứng"
    ],
    "Xét nghiệm": [
        "Xét nghiệm"
    ],
    "Sản phụ khoa": [
        "Sản phụ khoa", "Sức khỏe sinh sản", "Hỗ trợ sinh sản ivf", "Mang thai"
    ],
    "Máu": [
        "Máu", "Máu miễn dịch"
    ],
    "Ngoại lồng ngực": [
        "Ngoại lồng ngực"
    ]
}
    for department, keywords in departments.items():
        if label in keywords: return department

    return 0


In [128]:
df['Department'] = df['Department'].apply(lambda x:LabelUnifier(x))

In [129]:
df = df[~(df['Department'] == 0)]

In [130]:
df = df[['Symptom', 'Department', 'Category']]

In [133]:
df = df.reset_index(drop=True)

In [135]:
df.to_csv(fix_path, index=False)